<a href="https://colab.research.google.com/github/Geringer13/Netology_pyda/blob/master/Machine_learning_Homework_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0. Обозначаем корень дерева
1.Находим максимальный Gain (вспоминаем лекцию) для каждого признака.
2.Из полученных Gain’ов признаков выбираем максимальный, и таким образом определяется узел дерева.
3.Далее следуем по левому и правому разбиению, и выполняем пункты 1-3 до тех пор, пока Gain не станет равным 0. В этом случае, мы определяем лист дерева (то есть, возвращаемый ответ)

In [0]:
import numpy as np
from collections import Counter

def HEntropy(l):
    length = len(l)
    cnt = Counter(l)
    
    ent = 0
    for cl in cnt.values():
        p = cl / length
        l2 = np.log2(p)
        it = -p * l2
        ent += it
    
    return ent

def IG(H, l, i):
    left_l = l[:i]
    right_l = l[i:]
    return H(l) - (len(left_l) / len(l)) * H(left_l) - (len(right_l) / len(l)) * H(right_l)

In [0]:

import pandas as pd

class Splitter:
    '''
    Объект, умеющий определять, попадет ли конкретное измерение (item) в левую часть разбиения
    '''
    def __init__(self, attr_name, threshold_value):
        self.attr_name = attr_name
        self.threshold_value = threshold_value
        
    def split(self, item):
        return item[self.attr_name] < self.threshold_value
    
    def __str__(self):
        return f'{self.attr_name} < {self.threshold_value}'
        
class Node:
    def __init__(self, splitter, left_node, right_node):
#         print(f'Node created with splitter = {str(splitter)}' )
        self.splitter = splitter
        self.left_node = left_node
        self.right_node = right_node

    def evaluate(self, item):
        if self.splitter.split(item):
            return self.left_node.evaluate(item)
        else:
            return self.right_node.evaluate(item)
        
class Leaf:
    def __init__(self, clazz):
#         print(f'Leaf created for class = {clazz}' )
        self.clazz = clazz
        
    def evaluate(self, item):
        return self.clazz

In [0]:
def get_best_splitter(training_df, attr_name):
    """
    Parameters:
        trainig_df (pandas.DataFrame): датафрейм c учебной выборкой, в которой класс объекта находится в атрибуте clazz 
        attr_name атрибут, по которому ищем разбиение
    Returns:
        IG: значение Information Gain при наилучшем разбиении по атрибуту,
        Splitter: объект, умеющий определять, попадет ли конкретное измерение (item) в левую часть наилучшего разбиения
    """
    sorted_df = training_df.sort_values(attr_name)
    clazzes = sorted_df['clazz'].values
    max_ig = 0
    best_split_i = 0 
    for i in range(len(sorted_df)):
        ig = IG(HEntropy, clazzes, i)
        if ig > max_ig:
            max_ig = ig
            best_split_i = i
    
#     print(f'best_split_i={best_split_i} out of {len(sorted_df)-1}')
   
            
    best_split_value = sorted_df.iloc[best_split_i][attr_name]
#     print(f'best_split_value={best_split_value}')
    best_splitter = Splitter(attr_name, best_split_value)
    return max_ig, best_splitter


def build_tree(training_df, excluded_attrs=None):
    """
    Parameters:
        trainig_df (pandas.DataFrame): датафрейм c учебной выборкой, в которой класс объекта находится в атрибуте clazz 
    
    Returns:
        Node: корневой узел построенного дерева решений
    """
#     print(f'building node, len(training_df) = {len(training_df)}')
#     print(training_df['clazz'].unique())
    need_for_split = len(training_df['clazz'].unique()) > 1
    if not need_for_split:
        clazz = training_df.iloc[0]['clazz']
        return Leaf(clazz)
    
    if excluded_attrs == None:
        excluded_attrs = set(['clazz'])

    x_columns_names = [column for column in training_df.columns if column not in excluded_attrs]
    
    max_ig = 0
    best_splitter = None
    for attr_name in x_columns_names:
        ig, splitter = get_best_splitter(training_df, attr_name)
        if ig > max_ig:
            max_ig = ig
            best_splitter = splitter
    
#     print(f'max ig={max_ig}, splitter={str(best_splitter)}')
    
    left_df = training_df[training_df.apply(best_splitter.split, axis=1)]
    right_df = training_df[~training_df.apply(best_splitter.split, axis=1)]
#     print(f'len(left_df))={len(left_df)}')
#     print(f'len(right_df))={len(right_df)}')
    
    if len(left_df) == 0 or len(right_df) == 0:
#         print("bad split")
#         Такое бывает, когда из-за одинаковых значений атрибута у разных классов не получается отсечь один из них. 
#         Скорее всего лучше бы в этом случае просто создать Leaf с доминирующим классом, и это дало 
#         бы большую точность на тестовых данных, но по заданию нам надо разделить выборку до состояния, когда
#         в каждом листе находится не больше одного класса.
#         Поэтому просто построим текущий узел без учета атрибута, по котрому нельзя однозначно определить класс.
#         Конечно, если вдруг не окажется ни одного атрибута, по которому можно однозначно понять класс объекта,
#         то будет ошибка, но у нас тут учебное задание, не буду еще дальше усложнять код.
        excluded_attrs.add(best_splitter.attr_name)
        if(len(excluded_attrs) == len(training_df.columns)):
            raise Exception('Алярма! Ни по одному атрибуту нельзя однозначно разделить объекты разных классов')
        return build_tree(training_df, excluded_attrs)
    
    left_node = build_tree(left_df)
    right_node = build_tree(right_df)
    
    return Node(best_splitter, left_node, right_node)

In [0]:
from sklearn.datasets import load_iris
iris = load_iris()

training_df = pd.DataFrame(iris.data, columns=iris.feature_names)
training_df['clazz'] = pd.DataFrame(iris.target)

tree = build_tree(training_df)

In [18]:
def print_tree(node, indentation=''):
    print(indentation + "if " + str(node.splitter)+ ':')
    
    if isinstance(node.left_node, Node):
        print_tree(node.left_node, indentation + "\t")
    else:
        print(indentation + f"\tclass = {node.left_node.clazz}")
        
    print(indentation + f"else:")
    
    if isinstance(node.right_node, Node):
        print_tree(node.right_node, indentation + "\t")
    else:
        print(indentation + f"\tclass = {node.right_node.clazz}")

print_tree(tree)

if petal length (cm) < 3.0:
	class = 0.0
else:
	if petal width (cm) < 1.8:
		if petal length (cm) < 5.0:
			if petal width (cm) < 1.7:
				class = 1.0
			else:
				class = 2.0
		else:
			if petal length (cm) < 5.1:
				if sepal length (cm) < 6.7:
					class = 2.0
				else:
					class = 1.0
			else:
				if sepal length (cm) < 6.1:
					class = 1.0
				else:
					class = 2.0
	else:
		if sepal length (cm) < 5.9:
			class = 2.0
		else:
			if sepal width (cm) < 3.2:
				class = 2.0
			else:
				if sepal length (cm) < 6.2:
					class = 1.0
				else:
					class = 2.0


In [19]:
training_df['guess'] = training_df.apply(tree.evaluate, axis=1)
right_guesses = training_df[training_df['guess'] == training_df['clazz']]
accuracy = len(right_guesses)/len(training_df)
print(f'accuracy = {accuracy}')

accuracy = 1.0
